In [1]:
import numpy as np
import PWBeamformer as pwb
import matplotlib.pyplot as plt
import trig

In [2]:
dx = 0.1E-3
xmin = -10E-3
xmax = 10E-3
dy = 2E-3
ymin = 0E-3
ymax = 0E-3
dz = 0.06E-3
zmin = 1E-3
zmax = 11E-3

xfield = np.arange(xmin, xmax, dx)
yfield = 0
zfield = np.arange(zmin, zmax, dz)
Xfield, Yfield, Zfield = np.meshgrid(xfield, yfield, zfield)
field = np.array([Xfield.flatten(), Yfield.flatten(), Zfield.flatten()]).T

In [3]:
na = 11
nele = 128

params = {}
params['points'] = field
params['c'] = 1540
params['fnum'] = 1
params['alphas'] = np.repeat(np.linspace(-np.pi/8, np.pi/8, na), nele)
params['trefs']  = np.zeros(params['alphas'].shape)
params['refs'] = np.tile(1E-3 * np.array([[-5, 0, 0],[5, 0, 0]]), (na, 1))
params['ts'] = 1E-7
params['tstart'] = 0
params['nsamp'] = 1500

In [4]:
bmfrm = pwb.PWBeamformer(**params)

(33400, 3)